#### Notes:

- This demo will be on Databricks - first to show that MLflow is deeply integrated with Databricks
- Also hyperparameter tuning with Hyperopt did not work locally - the integrations are buggy and not set up correctly (online forums recommend just using Databricks for this)

###### Databricks 
- Free trial we pay for VMs on Azure but not for the Databricks service
- We need to use the Databricks ML Runtime, it has MLflow installed and integrated

### TODO Recording

- Go to portal.azure.com
- Search for Azure Databricks
- Create a Databricks workspace loony-db-ws (East US, loony-mlflow-rg, Free Trial)
- Launch the workspace
- On the top-left, click on the options for the 3 environments (Data Science, ML, SQL) 
- Pin and choose Machine Learning

##### Compute cluster
- Show the left pane with many options (we'll explore many of these as we do this demo)
- Click on Compute
- Create a new single-node cluster, call it loony-ml-cluster
- Choose a machine learning Databricks runtime (latest non-GPU runtime) 
- Create the cluster
- Wait till the cluster is up and running

##### Enabling DBFS
- Click on Data on the left - on the top right there is no option to Browse DBFS
- Click on the login on the top-right
- Go to Admin Settings -> Workspace Settings
- Enable DBFS
- Refresh the page
- Click on Data on the left - now you should see Browse DBFS
- Click on Browse DBFS and under FileStore/ upload the customer_travel.csv file

#### Experiments and Models
- Click on Experiments -> This is the experiments tab for MLflow should be empty now
- Click on Models -> This is where our registered models will show should be empty now

#### Workspace
- Click on workspace -> Users -> contact@loonycorn -> Import
- Import this notebook
- Open this notebook and connect to the loony-ml-cluster

In [0]:
import numpy as np
import pandas as pd
 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score

import mlflow
import mlflow.sklearn
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK

Loading the data
Displaying  the first 5 rows of the dataset
link=-https://www.kaggle.com/santoshd3/bank-customers?select=Churn+Modeling.csv

#### Notes

- The display() function is available in Databricks and gives a more interactive display

In [0]:
travel_churn_data = pd.read_csv('/dbfs/FileStore/customer_travel.csv')

travel_churn_data.display()

Age FrequentFlyer AnnualIncomeClass ServicesOpted AccountSyncedToSocialMedia BookedHotelOrNot Target 34 No Middle Income 6 No Yes 0 34 Yes Low Income 5 Yes No 1 37 No Middle Income 3 Yes No 0 30 No Middle Income 2 No No 0 30 No Low Income 1 No No 0 27 Yes High Income 1 No Yes 1 34 No Middle Income 4 Yes Yes 0 34 No Low Income 2 Yes No 1 30 No Low Income 3 No Yes 0 36 Yes High Income 1 No No 1 34 No Low Income 1 Yes Yes 0 28 No Middle Income 2 No No 1 35 No Record Middle Income 1 Yes Yes 0 34 Yes Low Income 4 No No 0 34 No Middle Income 5 No No 0 37 Yes Low Income 6 No Yes 0 30 No Low Income 1 Yes Yes 0 30 Yes High Income 1 Yes No 0 31 No Middle Income 1 No Yes 0 37 No Low Income 2 Yes No 1 30 No Middle Income 4 No Yes 0 31 Yes High Income 1 No No 1 34 Yes Low Income 1 Yes No 0 30 No Record Middle Income 2 No No 0 34 No Middle Income 1 No Yes 0 38 Yes Low Income 1 No Yes 0 37 No Middle Income 3 Yes No 0 30 No Middle Income 5 Yes No 0 28 No Low Income 1 Yes No 0 34 Yes High Income 1 No No 0 33 No Middle Income 6 No Yes 0 34 No Low Income 2 No No 0 27 No Middle Income 3 Yes No 0 35 Yes High Income 1 No No 1 30 No Record Low Income 4 No No 0 36 No Middle Income 2 No Yes 0 34 Yes Low Income 1 Yes Yes 0 37 Yes Low Income 1 Yes No 1 37 No Middle Income 3 No No 0 36 No Middle Income 2 No No 0 27 No Low Income 5 No Yes 0 36 Yes High Income 4 No No 0 28 No Middle Income 1 Yes Yes 0 30 No Low Income 2 No No 0 27 No Middle Income 3 No No 0 37 Yes High Income 6 No Yes 1 27 No Low Income 1 Yes No 0 38 No Middle Income 2 Yes No 0 30 No Middle Income 4 No Yes 0 34 Yes Low Income 1 No No 0 34 Yes Low Income 3 No Yes 1 31 No Middle Income 2 No No 0 34 No Low Income 1 Yes No 0 30 Yes High Income 5 No No 0 31 No Middle Income 1 No Yes 0 28 No Low Income 4 Yes Yes 1 30 No Record Middle Income 3 Yes Yes 0 37 Yes High Income 1 Yes No 1 36 No Low Income 1 No No 0 36 No Middle Income 2 No No 0 34 No Middle Income 6 No Yes 0 35 Yes Low Income 1 No No 0 30 No Middle Income 4 Yes No 0 29 No Middle Income 2 No No 0 33 Yes Low Income 1 Yes Yes 0 28 Yes High Income 1 No Yes 1 33 No Middle Income 5 Yes Yes 0 37 No Record Low Income 2 Yes No 1 31 No Middle Income 3 No No 0 34 Yes High Income 4 No No 1 37 No Low Income 1 No Yes 0 30 No Low Income 2 No No 0 30 No Middle Income 1 Yes Yes 0 30 Yes Low Income 1 Yes No 1 30 No Middle Income 3 No No 0 37 No Middle Income 6 No Yes 0 31 No Low Income 4 Yes No 0 34 Yes High Income 1 Yes No 0 34 Yes Low Income 1 No Yes 0 34 No Low Income 5 No No 0 28 No Middle Income 3 No Yes 0 27 Yes High Income 1 No No 1 30 No Low Income 1 Yes No 0 37 No Middle Income 4 No No 0 35 No Middle Income 1 No Yes 0 27 Yes Low Income 1 No Yes 1 35 No Middle Income 3 Yes No 0 30 No Middle Income 2 Yes No 0 37 No Low Income 1 No Yes 0 30 Yes High Income 1 No No 0 29 No Middle Income 6 No Yes 0 30 No Low Income 2 Yes No 1 30 No Low Income 5 Yes No 0 36 Yes High Income 1 No No 1 37 No Low Income 1 No No 0 28 No Middle Income 2 No Yes 1 30 No Middle Income 1 Yes Yes 0 31 Yes Low Income 4 Yes No 1 31 No Middle Income 3 No No 0 31 No Low Income 2 No No 0 30 No Record Low Income 1 Yes Yes 0 36 Yes High Income 1 No No 0 31 No Middle Income 1 Yes Yes 0 28 No Low Income 2 No No 1 30 No Middle Income 4 No Yes 0 30 Yes High Income 6 No Yes 1 37 Yes Low Income 1 Yes No 0 37 No Middle Income 2 Yes No 0 28 No Middle Income 1 No Yes 0 27 Yes Low Income 1 Yes No 1 34 No Middle Income 3 No Yes 0 30 No Record Middle Income 4 No No 0 33 No Low Income 1 Yes Yes 0 31 Yes High Income 1 No No 0 31 No Middle Income 1 No Yes 0 31 No Low Income 2 No Yes 0 30 No Middle Income 3 Yes No 0 34 Yes High Income 1 Yes No 1 34 No Low Income 5 Yes No 0 34 No Middle Income 2 No No 0 30 No Low Income 6 No Yes 0 28 Yes Low Income 1 No No 1 35 No Record Middle Income 3 Yes No 0 29 No Middle Income 2 No No 0 35 No Low Income 1 No No 0 31 Yes High Income 4 No Yes 0 35 No Middle Income 1 Yes Yes 0 29 No Low Income 2 Yes No 1 34 No Middle Income 3 No Yes 0 31 Yes High Income 1 No No 1 30

In [0]:
mapper = {'Low Income': 0, 'Middle Income': 1, 'High Income': 2}

travel_churn_data['AnnualIncomeClass'] = travel_churn_data['AnnualIncomeClass'].replace(mapper)

We are going to one hot encode categorical columns with dropping first category.

In [0]:
categorical_features = [
    'FrequentFlyer','AccountSyncedToSocialMedia', 'BookedHotelOrNot']

categorical_transformer = Pipeline(
    steps = [('encoder', OneHotEncoder(handle_unknown = 'ignore', drop = 'first'))])
    
preprocessor = ColumnTransformer(
    transformers = [("cat", categorical_transformer, categorical_features)], 
    remainder = StandardScaler()
)

Features data and target data are made into separate dataframes

In [0]:
X = travel_churn_data.drop(labels = ['Target'], axis = 1)
 
y = travel_churn_data['Target']
 
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 124)

### TODO Recording:

- Open Experiments in a new tab
- Create a new blank experiment hptuning_churn_prediction_model
- Show that the UI is the same as in local MLflow

In [0]:
import mlflow

mlflow.set_experiment(experiment_name = '/Users/contact@loonycorn.com/hptuning_churn_prediction_model')

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3495199824670206', creation_time=1687319814717, experiment_id='3495199824670206', last_update_time=1687319814717, lifecycle_stage='active', name='/Users/contact@loonycorn.com/hptuning_churn_prediction_model', tags={'mlflow.experiment.sourceName': '/Users/contact@loonycorn.com/hptuning_churn_prediction_model',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'contact@loonycorn.com',
 'mlflow.ownerId': '599420620358905'}>

Creating and train a decision tree classifier model and by using MLFLOW we are logging 
 parameters, metrics, and the model.

In [0]:
with mlflow.start_run():
   
    lr_model = LogisticRegression()
    
    pipe_lr = Pipeline( steps = [('preprocessor', preprocessor), ('classifier', lr_model)])
    pipe_lr.fit(X_train, y_train)

    predictions =  pipe_lr.predict(X_test) 
    predictions_predict_prob = pipe_lr.predict_proba(X_test)
    
    train_accuracy_score =  pipe_lr.score(X_train, y_train)
    test_accuracy_score = accuracy_score(y_test, predictions)
    test_precision_score = precision_score(y_test, predictions)
    test_recall_score = recall_score(y_test, predictions)
    test_f1_score = f1_score(y_test, predictions)
    auc_score = roc_auc_score(y_test,  predictions_predict_prob[:, 1])
    
    mlflow.log_params(lr_model.get_params())
    
    metrics = {
        'Train_accuracy_score': train_accuracy_score, 
        'Test_accuracy_score': test_accuracy_score,
        'Test_precision_score': test_precision_score,
        'Test_recall_score': test_recall_score,
        'Test_f1_score':test_f1_score,
        'AUC_score': auc_score 
    }
    
    mlflow.log_metrics(metrics)

    mlflow.sklearn.log_model(pipe_lr, 'lr_model')

/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


### TODO Recording

- Click on the "1 run" link
- Expand the sections on the run page and show that it is the same
- Show the artifacts -> the MLflow model is older 2.3

Hyperopt is a Python library for hyperparameter tuning. Databricks Runtime for Machine Learning includes an optimized and enhanced version of Hyperopt, including automated MLflow tracking.
Most of the code for a Hyperopt workflow is in the objective function.
We use the f1 score( As the dataset is quite highly imbalanced) as our objective parameter to compare the models' performance which is to be maximised 
Hyperopt tries to minimize the objective function. A F1 score means a better model, so you must return the negative f1 score.

When calling fmin(), Databricks recommends active MLflow run management; that is, wrap the call to fmin() inside a with mlflow.start_run(): statement. This ensures that each fmin() call is logged to a separate MLflow main run, and makes it easier to log extra tags, parameters, or metrics to that run.

In [0]:
from hyperopt.pyll import scope

def objective(params):
    with mlflow.start_run(nested = True):
            lr_model = LogisticRegression(C = params['C'], 
                                          solver = params['solver'],
                                          class_weight = params['class_weight'])
            
            pipe_lr = Pipeline(steps = [('preprocessor', preprocessor), ('model', lr_model)])
            pipe_lr.fit(X_train, y_train)
        
            predictions =  pipe_lr.predict(X_test) 
            predictions_predict_prob = pipe_lr.predict_proba(X_test)
    
            train_accuracy_score =  pipe_lr.score(X_train, y_train)
            test_accuracy_score = accuracy_score(y_test, predictions)
            test_precision_score = precision_score(y_test, predictions)
            test_recall_score = recall_score(y_test, predictions)
            test_f1_score = f1_score(y_test, predictions)
            auc_score = roc_auc_score(y_test,  predictions_predict_prob[:, 1])

            mlflow.log_params(lr_model.get_params())
            
            metrics = {
                'Train_accuracy_score': train_accuracy_score, 
               'Test_accuracy_score': test_accuracy_score,
               'Test_precision_score': test_precision_score,
               'Test_recall_score': test_recall_score,
               'Test_f1_score': test_f1_score,
               'AUC_score': auc_score 
            }
         
            mlflow.log_metrics(metrics)

            mlflow.sklearn.log_model(pipe_lr, 'lr_model')
  
            return {'loss': -test_f1_score, 'status': STATUS_OK}

Search space for hyperparameters is defined

In [0]:
search_space = {
    'C': hp.lognormal('C', 0, 1.0),
    'solver': hp.choice('solver', ['liblinear', 'lbfgs']),
    'class_weight':hp.choice('class_weight',[None, 'balanced'])
} 

Algorithm is defined

The two main choices are:

- hyperopt.tpe.suggest: Tree of Parzen Estimators, a Bayesian approach which iteratively and adaptively selects new hyperparameter settings to explore based on past results
- hyperopt.rand.suggest: Random search, a non-adaptive approach that samples over the search space

In [0]:
algo = tpe.suggest

Now We are running  the tuning algorithm with Hyperopt fmin()

- Setting  max_evals to the maximum number of points in hyperparameter space to test, that is, the maximum number of models to fit and evaluate, in our case it is set as 16.

In [0]:
with mlflow.start_run():
    best_lr_params = fmin(
        fn = objective,
        space = search_space,
        algo = algo,
        max_evals = 16)

print('Best value found: ', best_lr_params)


100%|██████████| 16/16 [00:56<00:00,  3.52s/trial, best loss: -0.4954128440366973]
Best value found:  {'C': 7.475529685261361, 'class_weight': 1, 'solver': 1}


#### TODO Recording

- After running the code above, stay on this page and watch as more runs are added to the same experiment
- Click on the Experiment icon on the right (showing a beaker)
- You can see a whole number of experiments logged there
- Open up the Experiments in a new tab (icon on the top right)

##### Experiment
- There should be 17 runs in the Experiment (one run is the parent and 16 child runs)
- Click on Columns and add columns for Test_f1_score, Test_accuracy_score, Test_AUC
- Sort by F1 score and find the run with the highest F1 score
- Click on that and expand the "Parameters" and "Metrics" tab and show
- IMPORTANT: Go to the experiments tab and delete all runs (so it's easier to see our next set of runs)

We are now using distributed tuning, adding one more argument to fmin(): a Trials class called SparkTrials.

SparkTrials takes 2 optional arguments:

- parallelism: Number of models to fit and evaluate concurrently. The default is the number of available Spark task slots.
- timeout: Maximum time (in seconds) that fmin() can run. The default is no maximum time limit.

This example uses the same  simple objective function defined earlier. In this case, the function runs quickly and the overhead of starting the Spark jobs dominates the calculation time, so the calculations for the distributed case take more time. For typical real-world problems, the objective function is more complex, and using SparkTrails to distribute the calculations will be faster than single-machine tuning.
Automated MLflow tracking is enabled by default.

Best value found out is with accuracy around 49% with parameters {'C': 1.462934164790394, 'class_weight': 1, 'solver': 0}

Now, we examine four algorithms available in scikit-learn: support vector machines (SVM), random forest, and logistic regression and Decison tree

In the following cell, we are defining  a parameter params['type'] for the model name. This function also runs the training and calculates the cross-validation accuracy.

In [0]:
def objective(params):
     with mlflow.start_run(nested = True):
         
        classifier_type = params['type']
        del params['type']
        
        if classifier_type == 'svm':
            clf = SVC(**params)
        elif classifier_type == 'rf':
            clf = RandomForestClassifier(**params)
        elif classifier_type == 'logreg':
            clf = LogisticRegression(**params)
        elif classifier_type == 'dtc':
            clf = DecisionTreeClassifier(**params)
        else:
            return 0
        
        pipeline = Pipeline(steps = [('preprocessor', preprocessor), ('model', clf)])
        
        pipeline.fit(X_train, y_train)
        
        predictions =  pipeline.predict(X_test) 
    
        train_accuracy_score =  pipeline.score(X_train, y_train)
        test_accuracy_score = accuracy_score(y_test, predictions)
        test_precision_score = precision_score(y_test, predictions)
        test_recall_score = recall_score(y_test, predictions)
        test_f1_score = f1_score(y_test, predictions)

        metrics = {
            'Train_accuracy_score': train_accuracy_score, 
            'Test_accuracy_score': test_accuracy_score,
            'Test_precision_score': test_precision_score,
            'Test_recall_score': test_recall_score,
            'Test_f1_score': test_f1_score 
        }

        mlflow.log_metrics(metrics)
        
        mlflow.sklearn.log_model(clf, 'clf_hpo')

        return {'loss': -test_f1_score, 'status': STATUS_OK}

Search space is defined for multiple models
Using hp.choice to select different models.

In [0]:
from hyperopt.pyll import scope

search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.lognormal('SVM_C', 0, 1.0),
        'kernel': hp.choice('kernel', ['linear', 'rbf'])
    },
    {
        'type': 'rf',
        'n_estimators':scope.int(hp.quniform('n_estimators', 100, 500, 50)),
        'max_depth': scope.int(hp.quniform('max_depth_rf', 4, 15, 1)),
        'min_samples_leaf': scope.int(hp.uniform('min_samples_leaf',1,5)),
        'min_samples_split': scope.int(hp.uniform('min_samples_split',2,6))
    },
    {
        'type': 'logreg',
        'C': hp.lognormal('LR_C', 0, 1.0),
        'solver': hp.choice('solver', ['liblinear', 'lbfgs']),
        'class_weight':hp.choice('class_weight',[None, 'balanced'])
    },
  
    {
        'type': 'dtc',
        'max_features':scope.int(hp.quniform('max_features', 1,10,1)),
        'max_depth': scope.int(hp.quniform('max_depth_dtc', 2, 20, 1)),
        'criterion': hp.choice('criterion_dtc', ['gini', 'entropy'])
    }
])

Same steps are repeated as done for single model case. Best parameters are {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 150, 'type': 'rf'} with f1 score around 0.73

In [0]:
algo = tpe.suggest

spark_trials = SparkTrials()

with mlflow.start_run():
    best_result = fmin(
        fn = objective, 
        space = search_space,
        algo = algo,
        max_evals = 32,
        trials = spark_trials
    )

Because the requested parallelism was None or a non-positive value, parallelism will be set to (4), which is Spark's default parallelism (4), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.
Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks. Click the 'stderr' link for a task to view trial logs.



100%|██████████| 32/32 [01:18<00:00,  2.46s/trial, best loss: -0.7073170731707318]


Total Trials: 32: 32 succeeded, 0 failed, 0 cancelled.


Using hyperopt.space_eval to display the results of the hyperparameter search.

In [0]:
import hyperopt

print(hyperopt.space_eval(search_space, best_result))

{'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 150, 'type': 'rf'}


#### TODO Recording

- Click on "experiment" and that will open up the Experiment on a new tab
- There should be 33 runs in the Experiment
- Sort by F1 score and find the run with the highest F1 score
- Click on that and expand the "Parameters" and "Metrics" tab and show